# Calculating Neural-Surprisal Correlation and Boundary Effects in the ECoG Dataset

This script computes the neural-surprisal correlation using the [open ECoG dataset](https://doi.org/10.1101/2025.02.14.638352).

The analysis involves two main steps:

- **Regressing out baseline auditory responses** using a temporal response function.
- **Calculating neural-surprisal correlation** for sentence-initial and non-initial words using Spearman correlation (with balanced sample sizes)

The computed results are stored in the `Calculated/` folder, and visualizations are provided in the `NeuralSurChunk_deenv_Vis.ipynb` notebook.  
Note: Due to randomization in the processing steps, results may vary slightly between runs. However, the main findings (e.g., chunking constrains prediction) remain consistent.

The preprocessing code is adapted from the open-source [podcast-ecog-tutorials](https://github.com/hassonlab/podcast-ecog-tutorials).  
We thank Zada et al. for publicly releasing the ECoG dataset and the tutorials that made this analysis possible.


## package

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nilearn.plotting import plot_markers
from mne_bids import BIDSPath

from himalaya.backend import set_backend, get_backend

import pickle

from scipy import stats 
from scipy import signal
from scipy.signal import find_peaks
from scipy.io import wavfile
np.random.seed(25)

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from himalaya.scoring import correlation_score
from himalaya.ridge import RidgeCV
# from voxelwise_tutorials.delayer import Delayer

# The Windows platform is not supported by the original voxelwise_tutorials package.
# Manually define the Delayer from the voxelwise_tutorials toolbox.
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted, check_array
class Delayer(BaseEstimator, TransformerMixin):
    """Scikit-learn Transformer to add delays to features.

    This assumes that the samples are ordered in time.
    Adding a delay of 0 corresponds to leaving the features unchanged.
    Adding a delay of 1 corresponds to using features from the previous sample.

    Adding multiple delays can be used to take into account the slow
    hemodynamic response, with for example `delays=[1, 2, 3, 4]`.

    Parameters
    ----------
    delays : array-like or None
        Indices of the delays applied to each feature. If multiple values are
        given, each feature is duplicated for each delay.

    Attributes
    ----------
    n_features_in_ : int
        Number of features seen during the fit.

    Example
    -------
    >>> from sklearn.pipeline import make_pipeline
    >>> from voxelwise_tutorials.delayer import Delayer
    >>> from himalaya.kernel_ridge import KernelRidgeCV
    >>> pipeline = make_pipeline(Delayer(delays=[1, 2, 3, 4]), KernelRidgeCV())
    """

    def __init__(self, delays=None):
        self.delays = delays

    def fit(self, X, y=None):
        """Fit the delayer.

        Parameters
        ----------
        X : array of shape (n_samples, n_features)
            Training data.

        y : array of shape (n_samples,) or (n_samples, n_targets)
            Target values. Ignored.

        Returns
        -------
        self : returns an instance of self.
        """
        X = self._validate_data(X, dtype='numeric')
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        """Transform the input data X, copying features with different delays.

        Parameters
        ----------
        X : array of shape (n_samples, n_features)
            Input data.

        Returns
        -------
        Xt : array of shape (n_samples, n_features * n_delays)
            Transformed data.
        """
        check_is_fitted(self)
        X = check_array(X, copy=True)

        n_samples, n_features = X.shape
        if n_features != self.n_features_in_:
            raise ValueError(
                'Different number of features in X than during fit.')

        if self.delays is None:
            return X

        X_delayed = np.zeros((n_samples, n_features * len(self.delays)),
                             dtype=X.dtype)
        for idx, delay in enumerate(self.delays):
            beg, end = idx * n_features, (idx + 1) * n_features
            if delay == 0:
                X_delayed[:, beg:end] = X
            elif delay > 0:
                X_delayed[delay:, beg:end] = X[:-delay]
            elif delay < 0:
                X_delayed[:-abs(delay), beg:end] = X[abs(delay):]

        return X_delayed

    def reshape_by_delays(self, Xt, axis=1):
        """Reshape an array, splitting and stacking across delays.

        Parameters
        ----------
        Xt : array of shape (n_samples, n_features * n_delays)
            Transformed array.
        axis : int, default=1
            Axis to split.

        Returns
        -------
        Xt_split :array of shape (n_delays, n_samples, n_features)
            Reshaped array, splitting across delays.
        """
        delays = self.delays or [0]  # deals with None
        return np.stack(np.split(Xt, len(delays), axis=axis))

# Model to restore punctuation in the text
from deepmultilingualpunctuation import PunctuationModel
punc_model = PunctuationModel('/data/model/oliverguhr/fullstop-punctuation-multilang-large')
# punc_model = PunctuationModel('F:/Model/oliverguhr/fullstop-punctuation-multilang-large')

backend = set_backend("numpy")
def FontSetting():
    plt.rcParams.update(
        {'font.size': 10,
          'font.family': ['Arial'],
          'font.weight': 'normal',
          # 'legend.fontsize': 'x-small',
          'legend.fontsize': 'medium',
          'axes.labelsize': 'Large',
          'axes.titlesize': 'Large',
          'axes.titleweight': 'normal',
          })
    plt.rcParams['svg.fonttype'] = 'none'
FontSetting()

d:\InstalledSoftware\Anaconda\envs\py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu
d:\InstalledSoftware\Anaconda\envs\py39\lib\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="none"` instead.
  warnings.warn(


## load features

### load surprisal

In [ ]:
bids_root = "YourPathToDataset" 
# ---------- contextual measures
# Load transcript and surprisal
transcript_path = f"{bids_root}stimuli/gpt2-xl/transcript.tsv"

df_contextual = pd.read_csv(transcript_path, sep="\t", index_col=0)
df_contextual['Surp'] = -np.log10(np.array(df_contextual['true_prob']) + 1e-9)
if "rank" in df_contextual.columns:
    model_acc = (df_contextual["rank"] == 0).mean()
    print(f"Model accuracy: {model_acc*100:.3f}%")


# words and correspd surprisal
df_word = df_contextual.groupby("word_idx").agg(dict(word="first", start="first", end="last"))

df_word_sur = df_contextual.groupby("word_idx").agg(dict(word="first", Surp="sum", entropy="first"))
df_word_sur.head()


Model accuracy: 30.942%


,word,Surp,entropy
word_idx,,,
0,Act,4.918057,2.402717
1,"one,",4.691066,3.732053
2,monkey,4.743413,6.621269
3,in,2.372939,4.444838
4,the,0.406486,3.781622


## Reading MEG and modeling

### functions

In [ ]:
def Epoching(raw, events):
    # epoch
    epochs = mne.Epochs(
        raw,
        events,
        tmin=-2,
        tmax=2,
        baseline=None,
        proj=False,
        event_id=None,
        preload=True,
        event_repeated="merge",
    )
    
    print(f"Epochs object has a shape of: {epochs._data.shape}")
    return epochs

def DataReading(subj, bids_root, df_word, fs_re):
    file_path = BIDSPath(root=f"{bids_root}derivatives/ecogprep",
                        subject=subj, 
                        task="podcast", datatype="ieeg", description="highgamma",
                        suffix="ieeg", extension=".fif")
    print(f"File path within the dataset: {file_path}")

    # You only need to run this if using Colab (i.e. if you did not set bids_root to a local directory)
    if not len(bids_root):
        !wget -nc https://s3.amazonaws.com/openneuro.org/ds005574/$file_path
        file_path = file_path.basename

    # data loading
    raw = mne.io.read_raw_fif(file_path, verbose=False)
    print(len(raw.ch_names))

    # get event
    events = np.zeros((len(df_word), 3), dtype=int)
    events[:, 0] = (df_word.start * raw.info['sfreq']).astype(int)
    print(events.shape)

    # resample
    raw, events = raw.resample(fs_re, events=events)

    # epoch
    epochs = Epoching(raw, events)
    
    return raw, epochs, events
    
def Boundaries(punc_model, epochs, df_word_sur):
    # recover punc and label boundaries
    pass_str = ' '.join(df_word_sur.iloc[epochs.selection].word)
    result = punc_model.restore_punctuation(pass_str)
    word_punc = result.split(' ')
    ini_idx = np.array([tmp_i+1 for tmp_i, tmp in enumerate(word_punc) 
                        if tmp.endswith((',', '!', '.', '?', ':'))][:-1])
    non_ini_idx = np.array([tmp_i+1 for tmp_i, tmp in enumerate(word_punc) 
                            if not tmp.endswith((',', '!', '.', '?', ':'))][:-1])

    return ini_idx, non_ini_idx

def ModelInit(fs_):
    lambdas = [1e-20] 
    # In this analysis, regularization does not improve modeling performance.
    # Instead, it tends to shrink the predicted signal, making the acoustic responses
    # remain in the residuals. Therefore, we set the regularization parameter 
    # to a very small value. 
    inner_cv = KFold(n_splits=5, shuffle=False)
    # ridge regression
    scaler = StandardScaler(with_mean=False, with_std=True)
    delayer = Delayer(delays=np.arange(-int(fs_*1.5), int(fs_*1.5)+1))
    solver_params = dict(n_alphas_batch=10)

    ridge = RidgeCV(
        alphas=lambdas, cv=inner_cv, 
        solver_params=solver_params, fit_intercept=True)
    pipeline = Pipeline([
        ('scaler', scaler),
        ('delayer', delayer),
        ('ridgecv', ridge)
        ])
    return pipeline

def TRFRegre(X, Y, fs_):
    # X: time x feas
    # Y: time x outputs
    outer_cv = KFold(n_splits=2)
    h_hat_s = []
    corrs = []
    for train, test in outer_cv.split(Y):
        pipeline = ModelInit(fs_)
        pipeline.fit(X[train], Y[train])
        Y_hat = pipeline.predict(X[test])
        # cc
        corr_tmp = correlation_score(Y[test], Y_hat)
        corrs.append(corr_tmp)

        # waveform
        ridge_fitted = pipeline['ridgecv']

        h_hat = np.transpose(
            np.reshape(ridge_fitted.coef_, 
            (X.shape[-1], -1, Y.shape[-1]), order='F'),
            [1, 0, 2])
    
        h_hat_s.append(h_hat)
    cc = np.stack(corrs).mean(0)
    return cc, h_hat_s, pipeline


def discrete_envelope(envelope):
    peaks, _ = find_peaks(envelope)
    gradient = np.gradient(envelope, 1/100)
    gradient[gradient<0]=0
    peaks_grad,_ = find_peaks(gradient)

    vec_peaks = np.zeros(len(gradient))
    vec_peaks[peaks] = envelope[peaks]

    vec_grad = np.zeros(len(gradient))
    vec_grad[peaks_grad] = gradient[peaks_grad]
    final_peaks = vec_peaks
    final_gradient = vec_grad
    return final_peaks, final_gradient

def get_env(fs_re=32):
    audio_path = f"{bids_root}stimuli/podcast.wav"
    highfs, highqa = wavfile.read(audio_path)
    if highqa.ndim > 1:
        highqa = highqa[:, 0]  # take first channel
    env = signal.resample(abs(highqa), num=round(highqa.size / highfs * fs_re))

    env_peaks, env_gradient = discrete_envelope(env)

    return env, env_peaks, env_gradient

def TRFControlAcoustic(raw, epochs, events, fs_re, subj, non_ini_idx, ini_idx):
    # ===regress out envelope and word boundaries
    f_tmp1, axes_tmp1 = plt.subplots(2, 3, layout="constrained", figsize=(8, 6))

    # ERP: before regression
    epochs[non_ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[0, 0])
    epochs[ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[1, 0])
    
    # signal
    cont_data = raw.get_data()
    # features
    env, env_peaks, env_gradient = get_env(fs_re)
    w_onset = np.zeros(env.shape)
    s_onset = np.zeros(env.shape)

    w_onset_idx = events[epochs.selection]
    w_onset[w_onset_idx[non_ini_idx]] = 1

    s_onset_idx = w_onset_idx[ini_idx]
    s_onset[s_onset_idx] = 1

    contr_feas = np.array([env_peaks, env_gradient, w_onset, s_onset])

    # modeling
    print('===================== Modeling ==================')
    cc, h_hat_s, pipe_trf_simu = TRFRegre(
        contr_feas.T, cont_data.T, 
        fs_re)
    cont_data_hat_T = pipe_trf_simu.predict(contr_feas.T).T

    # predicted signal
    print('===================== Visualization ==================')
    raw1 = mne.io.RawArray(cont_data_hat_T, raw.info)
    epochs_pred = Epoching(raw1, events)
    epochs_pred[non_ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[0, 1])
    epochs_pred[ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[1, 1])

    # residual
    resi_data = cont_data - cont_data_hat_T

    raw2 = mne.io.RawArray(resi_data, raw.info)
    epochs_resi = Epoching(raw2, events)
    epochs_resi[non_ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[0, 2])
    epochs_resi[ini_idx].average().apply_baseline(baseline=(-2, -1)).plot(
        show=False, axes=axes_tmp1[1, 2])

    # TRF
    h_hat_s = backend.to_numpy(h_hat_s)
    h_hat_s_mean = np.stack(h_hat_s).mean(axis=0)
    f_tmp2, axs_tmp2 = plt.subplots(1, h_hat_s_mean.shape[1], figsize=(10, 3))
    for fea_i in range(h_hat_s_mean.shape[1]):
        evoked_trf = mne.EvokedArray(h_hat_s_mean[:,fea_i,:].T, raw.info, tmin=-1.5)
        evoked_trf.plot(show=False, axes=axs_tmp2[fea_i])
    plt.tight_layout()
    plt.show()

    f_tmp1.savefig(f'./intermediate_plot/TRF_signal_{subj}.png')
    f_tmp2.savefig(f'./intermediate_plot/TRF_{subj}.png')

    raw._data = resi_data
    epochs = Epoching(raw, events)

    return raw, epochs

def SignalSurp(epochs, df_word_sur):
    epochs_data = epochs.get_data() # electrodes * number of lags
    epochs_data = epochs_data.reshape((len(epochs), -1), order='F')
    Y = epochs_data

    if "torch" in get_backend().__name__:
        Y = Y.astype(np.float32)

    # surprisal
    X_s = df_word_sur.iloc[epochs.selection].loc[:, ['Surp']].values
    return Y, X_s

def closest_elements(A, B):
    diff_tmp = abs(A[:, np.newaxis] - B[np.newaxis, :])
    best_idx = []
    for base_i in range(len(B)):
        min_r_tmp = np.nanargmin(diff_tmp[:, base_i])
        diff_tmp[min_r_tmp, :] = np.nan
        best_idx.append(min_r_tmp)
    best_idx = np.array(best_idx)

    return best_idx

def CorrelationAna(X_s, Y, non_ini_idx, ini_idx):
    se_i = 0
    # correlation
    cc_surp = [stats.spearmanr(X_s[:, se_i], tmp)[0] for tmp in Y.T]

    # For noninitial words: randomly select a subset to balance the sample size.
    non_ini_idx_bala = np.random.choice(non_ini_idx, (101, ini_idx.shape[0]))
    cc_non_surp = [
        [stats.spearmanr(X_s[tmp_idx, se_i], tmp[tmp_idx])[0] 
         for tmp in Y.T]
         for tmp_idx in non_ini_idx_bala
         ]
    # We also sampled non-initial words to balance the surprisal distribution 
    # between initial and non-initial words. The boundary effect remains similar 
    # when the surprisal distribution is balanced.
    distr_idx = closest_elements(X_s[non_ini_idx, se_i], X_s[ini_idx, se_i])
    cc_non_surp_distr = [
        stats.spearmanr(X_s[non_ini_idx, se_i][distr_idx], tmp[non_ini_idx][distr_idx])[0] 
        for tmp in Y.T]
    
    # for initial words
    cc_ini_surp = [stats.spearmanr(X_s[ini_idx, se_i], tmp[ini_idx])[0] for tmp in Y.T]
    cc_surp = np.array([cc_surp]+cc_non_surp+[cc_non_surp_distr]+[cc_ini_surp]).T

    return cc_surp

### Main script

In [ ]:
subjs = ["01", "02", "03", "04", "05", "06", "07", "08", "09"]
fs_re = 32
f_subjs, axes = plt.subplots(2, len(subjs), layout="constrained", figsize=(20, 5))
for subj_i, subj in enumerate(subjs):
    print(f'********** subj: {subj}, Dataloading **************')
    raw, epochs, events = DataReading(
        subj, bids_root, df_word, fs_re)

    print(f'********** subj: {subj}, Regress out envelope **************')
    ini_idx, non_ini_idx = Boundaries(punc_model, epochs, df_word_sur)

    raw, epochs = TRFControlAcoustic(
        raw, epochs, events, fs_re, subj, non_ini_idx, ini_idx)
    
    print(f'********** subj: {subj}, Preparing features for modeling **************')
    Y, X_s = SignalSurp(epochs, df_word_sur)

    print(f'********** subj: {subj}, Neural-Surprisal correlation **************')
    cc_surp = CorrelationAna(X_s, Y, non_ini_idx, ini_idx)

    # saving....
    ch_name = [ch['ch_name'] for ch in raw.info['chs']]
    ch_name = np.array(ch_name)

    coords = np.vstack([ch['loc'][:3] for ch in raw.info['chs']])
    coords *= 1000  # nilearn likes to plot in meters, not mm
    coords = np.array(coords)

    data_saving = {'cc_surp':cc_surp, 'ch_name':ch_name, 'coords':coords}
    with open(f"./Calculated/NeuralSurp_{subj}.pickle", "wb") as output_file:
        pickle.dump(data_saving, output_file)
plt.show()
f_subjs.savefig('./intermediate_plot/ERP.png', dpi=300)